# In-situ prepration for Classification

In [1]:
import glob, os
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

import rasterstats
from rasterstats import zonal_stats
from pathlib import Path
from IPython.display import display

print(f'Pandas    : {pd.__version__}')
print(f'GeoPandas : {gpd.__version__}')

Pandas    : 1.1.5
GeoPandas : 0.8.1


## Set paths for input and output directories

Create directories if there are missing using `Path` and `mkdir`

In [2]:
grp_letter   = 'X'
student_name = 'ndeffense'

# When you are connected to the computer room

vector_path = 'X:/data/VECTOR/'
raster_path = 'X:/data/RASTER/'
output_path = f'X:/GROUP_{grp_letter}/TP/{student_name}/DATA/'


print(f'Vector input path are set to : {vector_path}')
print(f'Raster input path are set to : {raster_path}')
print(f'Output path are set to       : {output_path}')

Vector input path are set to : X:/data/VECTOR/
Raster input path are set to : X:/data/RASTER/
Output path are set to       : X:/GROUP_X/TP/ndeffense/DATA/


## 1. Prepare in-situ dataset

<img src="figures/prepare_in_situ.png" width="10000">

### Set up filenames and parameters

In [5]:
# In-Situ datasets
in_situ_shp = f'{vector_path}WALLONIA_2018_IN_SITU_ROI.shp'

gdf = gpd.read_file(in_situ_shp)

# Raster template with no NaN (before applying SCL)
img_temp_tif = f'{raster_path}IM_ROI/T31UFR_20200316T104709_B02_10m_ROI.tif'


# Parameters
# ----------

buf_size = -10

display(gdf.head())

,ID,CROP,LC,CODE,IRRIGATION,geometry
0,1877,1,Maize (for livestock),201,0,"POLYGON ((634719.076 5591248.019, 634788.109 5..."
1,1878,1,Common wheat (winter),311,0,"POLYGON ((634996.603 5591774.690, 635051.974 5..."
2,1879,1,Maize (for livestock),201,0,"POLYGON ((635196.857 5591331.590, 635199.724 5..."
3,1880,1,Grassland (temporary),62,0,"POLYGON ((635003.339 5591080.041, 635021.807 5..."
4,3251,1,Barley (winter),321,0,"POLYGON ((634708.850 5590509.262, 634710.400 5..."


### 1.1 Apply a negative buffer

This step is useful to avoid border effects

In [8]:
gdf.geometry = gdf.geometry.buffer(buf_size)

gdf = gdf[~gdf.geometry.is_empty]    # Remove empty geometries

display(gdf)

,ID,CROP,LC,CODE,IRRIGATION,geometry
0,1877,1,Maize (for livestock),201,0,"POLYGON ((634753.108 5591233.868, 634791.680 5..."
1,1878,1,Common wheat (winter),311,0,"POLYGON ((635019.340 5591754.362, 635051.688 5..."
2,1879,1,Maize (for livestock),201,0,"POLYGON ((635217.057 5591352.077, 635219.722 5..."
3,1880,1,Grassland (temporary),62,0,"MULTIPOLYGON (((635030.387 5591072.662, 635031..."
4,3251,1,Barley (winter),321,0,"POLYGON ((634729.316 5590538.110, 634730.397 5..."
...,...,...,...,...,...,...
368,183201,0,Not agriculture,6,0,"POLYGON ((628782.385 5595564.680, 628782.231 5..."
370,183202,0,Not agriculture,9,0,"POLYGON ((628111.183 5595425.145, 628111.877 5..."
371,183206,0,Not agriculture,19,0,"POLYGON ((627773.807 5595677.024, 627773.877 5..."
373,183210,0,Not agriculture,6,0,"POLYGON ((631948.730 5595870.384, 631949.003 5..."


### 1.1.1 Explode *MultiPolygon* geometry into individual *Polygon* geometries using `GeoPandas` and `Shapely`

*MultiPolygons* geometries can cause problems later on (e.g. for rasterisation) !

In [9]:
# Create a function to explode MultiPolygons from a GeoDataFrame
def explode(indf):
    outdf = gpd.GeoDataFrame(columns=indf.columns)
    for idx, row in indf.iterrows():
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns=indf.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
    return outdf


# Apply function on GeoDataFrame
gdf = explode(gdf)

display(gdf)

,ID,CROP,LC,CODE,IRRIGATION,geometry
0,1877,1,Maize (for livestock),201,0,"POLYGON ((634753.108 5591233.868, 634791.680 5..."
1,1878,1,Common wheat (winter),311,0,"POLYGON ((635019.340 5591754.362, 635051.688 5..."
2,1879,1,Maize (for livestock),201,0,"POLYGON ((635217.057 5591352.077, 635219.722 5..."
3,1880,1,Grassland (temporary),62,0,"POLYGON ((635030.387 5591072.662, 635031.156 5..."
4,1880,1,Grassland (temporary),62,0,"POLYGON ((635178.887 5590784.638, 635180.083 5..."
...,...,...,...,...,...,...
346,183201,0,Not agriculture,6,0,"POLYGON ((628782.385 5595564.680, 628782.231 5..."
347,183202,0,Not agriculture,9,0,"POLYGON ((628111.183 5595425.145, 628111.877 5..."
348,183206,0,Not agriculture,19,0,"POLYGON ((627773.807 5595677.024, 627773.877 5..."
349,183210,0,Not agriculture,6,0,"POLYGON ((631948.730 5595870.384, 631949.003 5..."


### 1.2 Add a column with area of each polygons

In [11]:
gdf['area'] = gdf.geometry.area.astype(int)

display(gdf.head())

,ID,CROP,LC,CODE,IRRIGATION,geometry,area
0,1877,1,Maize (for livestock),201,0,"POLYGON ((634753.108 5591233.868, 634791.680 5...",17186
1,1878,1,Common wheat (winter),311,0,"POLYGON ((635019.340 5591754.362, 635051.688 5...",47779
2,1879,1,Maize (for livestock),201,0,"POLYGON ((635217.057 5591352.077, 635219.722 5...",16887
3,1880,1,Grassland (temporary),62,0,"POLYGON ((635030.387 5591072.662, 635031.156 5...",3272
4,1880,1,Grassland (temporary),62,0,"POLYGON ((635178.887 5590784.638, 635180.083 5...",3498


### 1.3 Add pixel count to in-situ data

#### 1.3.1 Zonal Stats using `rasterstats` and store output in a DataFrame

In [13]:
zs_df = pd.DataFrame(zonal_stats(vectors=gdf,
                                 raster=img_temp_tif,
                                 nodata=-999,
                                 stats='count'))

zs_df = zs_df.rename(columns={'count': 'pix_count'})

display(zs_df.head())

# Join pixels count with polygons informations

pixCount_gdf = pd.concat([gdf, zs_df], axis=1, join="inner")

display(pixCount_gdf.head())

,pix_count
0,172
1,472
2,169
3,33
4,33


,ID,CROP,LC,CODE,IRRIGATION,geometry,area,pix_count
0,1877,1,Maize (for livestock),201,0,"POLYGON ((634753.108 5591233.868, 634791.680 5...",17186,172
1,1878,1,Common wheat (winter),311,0,"POLYGON ((635019.340 5591754.362, 635051.688 5...",47779,472
2,1879,1,Maize (for livestock),201,0,"POLYGON ((635217.057 5591352.077, 635219.722 5...",16887,169
3,1880,1,Grassland (temporary),62,0,"POLYGON ((635030.387 5591072.662, 635031.156 5...",3272,33
4,1880,1,Grassland (temporary),62,0,"POLYGON ((635178.887 5590784.638, 635180.083 5...",3498,33


### 1.4 Write dataset in shapefiles

In [15]:
in_situ_count_shp = f'{output_path}WALLONIA_2018_IN_SITU_pixCount.shp'

pixCount_gdf.to_file(in_situ_count_shp)

## 2. Sampling Design


<img src="figures/in_situ_sampling_design.png" width="10000">

### Set up filenames and parameters

In [18]:
in_situ_cal_shp = f'{output_path}IN_SITU/WALLONIA_2018_IN_SITU_ROI_cal.shp'
in_situ_val_shp = f'{output_path}IN_SITU/WALLONIA_2018_IN_SITU_ROI_val.shp'

csv_strategies = f'{output_path}IN_SITU/WALLONIA_2018_IN_SITU_ROI_sampling_design.csv'

# Open the in-situ shapefile in a GeoDataFrame

parcels_gdf = gpd.read_file(in_situ_count_shp)
nb_poly_ori = len(parcels_gdf)
print(f'----> There are {nb_poly_ori} polygons')

# Parameters
# ----------

S2pixMin         = 3
S2pixBest        = 10
S2pixThres       = 10000
SampleRatioCal_1 = 0.25
SampleRatioCal_2 = 0.75

----> There are 375 polygons


### 2.1 Remove parcels with not enough pixels

In [19]:
print(f'-- Remove parcels with less than {S2pixMin} pixels')

parcels_gdf = parcels_gdf.loc[parcels_gdf['pix_count'] >= S2pixMin]

nb_remove_small_poly = len(parcels_gdf)

print(f'----> There are {nb_remove_small_poly} polygons with enough pixels')

-- Remove parcels with less than 3 pixels
----> There are 341 polygons with enough pixels


### 2.2 Select best parcels

In [20]:
print(f'-- Move parcels with less than {S2pixBest} pixels in the validation dataset')

parcels_non_best_gdf = parcels_gdf.loc[parcels_gdf['pix_count'] < S2pixBest]
parcels_best_gdf = parcels_gdf.loc[parcels_gdf['pix_count'] >= S2pixBest]

print(f'----> {len(parcels_best_gdf)} best parcels')
print(f'----> {len(parcels_non_best_gdf)} non-best parcels (moved in the validation dataset)')

-- Move parcels with less than 10 pixels in the validation dataset
----> 324 best parcels
----> 17 non-best parcels (moved in the validation dataset)


### 2.3 Split parcels into calibration and validation datasets
#### 2.3.1 Create a table with the number of pixels to get for calibration and validation (depending on the strategy)

In [21]:
print('-- Split parcels into cal/val')

pixel_per_class_df = parcels_best_gdf.groupby('CODE')['pix_count'].agg('sum').to_frame().reset_index()

pixel_per_class_df['strategy'] = np.where(pixel_per_class_df['pix_count'] >= S2pixThres, 1, 2)


classes_strategy_1 = pixel_per_class_df.loc[np.where(pixel_per_class_df['strategy'] == 1)]
classes_strategy_2 = pixel_per_class_df.loc[np.where(pixel_per_class_df['strategy'] == 2)]


# Strategy 1
# ----------

classes_strategy_1['pix_cal'] = classes_strategy_1['pix_count']*SampleRatioCal_1
classes_strategy_1['pix_val'] = classes_strategy_1['pix_count']*(1-SampleRatioCal_1)

# Strategy 2
# ----------

classes_strategy_2['pix_cal'] = classes_strategy_2['pix_count']*SampleRatioCal_2
classes_strategy_2['pix_val'] = classes_strategy_2['pix_count']*(1-SampleRatioCal_2)


classes_strategy_all = pd.concat([classes_strategy_1, classes_strategy_2]).reset_index()

display(classes_strategy_all)

-- Split parcels into cal/val


,index,CODE,pix_count,strategy,pix_cal,pix_val
0,5,311,25092,1,6273.00,18819.00
1,16,62,13499,1,3374.75,10124.25
2,22,91,10833,1,2708.25,8124.75
3,0,12,71,2,53.25,17.75
4,1,19,1758,2,1318.50,439.50
5,2,20,146,2,109.50,36.50
6,3,201,6293,2,4719.75,1573.25
7,4,21,208,2,156.00,52.00
8,6,321,6200,2,4650.00,1550.00
9,7,341,329,2,246.75,82.25


#### 2.3.2 Select randomly parcels to reach the number of pixels needed to calibrate the model

In [22]:
parcels_cal_gdfs = []
parcels_val_gdfs = []

for i in range(0,len(classes_strategy_all)):

    code = classes_strategy_all.loc[i,'CODE']
    strategy  = classes_strategy_all.loc[i,'strategy']

    pixel_cal = int(classes_strategy_all['pix_cal'].loc[classes_strategy_all['CODE'] == code])

    parcels_by_sub_class = parcels_best_gdf.loc[parcels_best_gdf['CODE'] == code]
    parcels_by_sub_class_reordered = parcels_by_sub_class.sample(len(parcels_by_sub_class), random_state=10)

    parcels_cal_gdf = parcels_by_sub_class_reordered[parcels_by_sub_class_reordered['pix_count'].cumsum() <= pixel_cal]
    parcels_val_gdf = parcels_by_sub_class_reordered[parcels_by_sub_class_reordered['pix_count'].cumsum() > pixel_cal]

    if pixel_cal < int(parcels_cal_gdf['pix_count'].sum()):
        print("error")

    parcels_cal_gdfs.append(parcels_cal_gdf)
    parcels_val_gdfs.append(parcels_val_gdf)


# Add non best parcels to validation
parcels_val_gdfs.append(parcels_non_best_gdf)

parcels_cal_final_gdf = pd.concat(parcels_cal_gdfs, ignore_index=True)
parcels_val_final_gdf = pd.concat(parcels_val_gdfs, ignore_index=True)


print(f'----> {len(parcels_cal_final_gdf)} parcels are in calibration dataset')
print(f'----> {len(parcels_val_final_gdf)} parcels are in validation dataset')

----> 167 parcels are in calibration dataset
----> 174 parcels are in validation dataset


### 2.4 Write datasets in shapefiles

In [23]:
parcels_cal_final_gdf.to_file(in_situ_cal_shp)
parcels_val_final_gdf.to_file(in_situ_val_shp)

classes_strategy_all.to_csv(csv_strategies, index=False, sep=';')